In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import norm
import datetime
import warnings
warnings.filterwarnings('ignore')

In [12]:
df = pd.read_csv('us-counties.csv')
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [13]:
df2 = pd.read_csv('train_2016.csv')
df2.head()

,FIPS,County,DEM,GOP,MedianIncome,MigraRate,BirthRate,DeathRate,BachelorRate,UnemploymentRate
0,18019,"Clark County, IN",18791,30012,"51,837",4.9,12.8,11.0,20.9,4.2
1,6035,"Lassen County, CA",2026,6533,"49,793",-18.4,9.2,6.3,12.0,6.9
2,40081,"Lincoln County, OK",2423,10838,"44,914",-1.3,11.4,11.7,15.1,5.3
3,31153,"Sarpy County, NE",27704,44649,"74,374",9.2,14.2,5.0,40.1,2.9
4,28055,"Issaquena County, MS",395,298,"26,957",-12.8,9.8,5.3,6.7,14.0


In [14]:
df3 = pd.read_csv('test_2016_no_label.csv')
df3.head()

,FIPS,County,MedianIncome,MigraRate,BirthRate,DeathRate,BachelorRate,UnemploymentRate
0,17059,"Gallatin County, IL","39,634",-10.6,10.8,12.5,9.9,7.6
1,6103,"Tehama County, CA","40,585",1.8,12.8,10.4,15.5,7.1
2,42047,"Elk County, PA","49,274",-9.3,9.7,13.0,18.6,5.5
3,47147,"Robertson County, TN","58,487",7.4,12.7,9.7,18.6,4.1
4,39039,"Defiance County, OH","52,210",-5.3,11.1,10.2,16.7,4.9


In [25]:
df5 = pd.concat([df2.drop(['DEM', 'GOP'], axis = 1), df3])
df5.head()

,FIPS,County,MedianIncome,MigraRate,BirthRate,DeathRate,BachelorRate,UnemploymentRate
0,18019,"Clark County, IN","51,837",4.9,12.8,11.0,20.9,4.2
1,6035,"Lassen County, CA","49,793",-18.4,9.2,6.3,12.0,6.9
2,40081,"Lincoln County, OK","44,914",-1.3,11.4,11.7,15.1,5.3
3,31153,"Sarpy County, NE","74,374",9.2,14.2,5.0,40.1,2.9
4,28055,"Issaquena County, MS","26,957",-12.8,9.8,5.3,6.7,14.0


In [16]:
WY = pd.read_csv('WyomingUnemployment.csv')
WA = pd.read_csv('WashUnemployment.csv')
VA = pd.read_csv('VirgUnemployment.csv')
OH = pd.read_csv('OhioUnemployment.csv')
WY.head()

,fips1,name,year,UR,year.1,UR.1,year.2,UR.2,year.3,UR.3,...,year.5,UR.5,year.6,UR.6,year.7,UR.7,year.8,UR.8,Median Household Income 2018,% State Median
0,56000,Wyoming,2011,5.8,2012,5.3,2013,4.7,2014,4.1,...,2016,5.3,2017,4.2,2018,3.9,2019,3.6,"$62,259",100.00%
1,56001,"Albany County, WY",2011,4.2,2012,4.3,2013,3.8,2014,3.4,...,2016,3.2,2017,2.9,2018,3.2,2019,3.1,"$47,561",76.40%
2,56003,"Big Horn County, WY",2011,6.1,2012,5.6,2013,5.2,2014,4.5,...,2016,5.0,2017,4.2,2018,4.5,2019,4.2,"$55,195",88.70%
3,56005,"Campbell County, WY",2011,4.9,2012,4.6,2013,4.2,2014,3.3,...,2016,7.2,2017,4.8,2018,3.9,2019,3.7,"$78,112",125.50%
4,56007,"Carbon County, WY",2011,6.0,2012,5.2,2013,4.3,2014,4.0,...,2016,4.7,2017,4.0,2018,3.6,2019,3.3,"$61,773",99.20%


In [21]:
def preprocess(state, us):
    name = state['name'][0]
    us = us[us['state'] == name]
    state['fips1'] = state['fips1'].astype(float)
    fips = state['fips1']
    for i, code in enumerate(fips):
        fips[i] = sum(us[us['fips'] == code]['cases'])
    state['Cases'] = np.array(fips)
    state = state.drop(['year', 'year.1', 'year.2', 'year.3', 'year.4', 'year.5', 'year.6', 'year.8', 'UR', 'UR.1','UR.2', 'UR.3', 'UR.4', 'UR.5', 'UR.6', 'UR.8'], axis=1)
    money = state['Median Household Income 2018']
    incomes = []
    for income in money:
        incomes.append(float(income.split(',')[0][1:] + income.split(',')[1]))
    state['Median Household Income 2018'] = incomes
    return state
joinedWY = preprocess(WY.copy(), df.copy())
joinedWA = preprocess(WA.copy(), df.copy())
joinedVA = preprocess(VA.copy(), df.copy())
joinedOH = preprocess(OH.copy(), df.copy())

# fig, axs = plt.subplots(2,2, figsize=(10,10))
# axs[0,0].scatter(joinedOH['Median Household Income 2018'][1:], joinedOH['Cases'][1:])
# axs[0,0].set_title("Ohio")
# axs[0,1].scatter(joinedVA['Median Household Income 2018'][1:], joinedVA['Cases'][1:])
# axs[0,1].set_title("Virginia")
# axs[1,0].scatter(joinedWA['Median Household Income 2018'][1:], joinedWA['Cases'][1:])
# axs[1,0].set_title("Washington")
# axs[1,1].scatter(joinedWY['Median Household Income 2018'][1:], joinedWY['Cases'][1:])
# axs[1,1].set_title("Wyoming")
# for ax in axs.flat:
#     ax.set(xlabel="Median Household Income (2018)", ylabel="Cases")

In [22]:
def preprocess2(state, us):
    name = state['name'][0]
    us = us[us['state'] == name]
    state['fips1'] = state['fips1'].astype(float)
    fips = state['fips1']
    for i, code in enumerate(fips):
        fips[i] = sum(us[us['fips'] == code]['deaths'])/sum(us[us['fips'] == code]['cases']) if sum(us[us['fips'] == code]['cases']) != 0 else 0
    state['Mortality'] = np.array(fips)
    state = state.drop(['year', 'year.1', 'year.2', 'year.3', 'year.4', 'year.5', 'year.6', 'year.8', 'UR', 'UR.1','UR.2', 'UR.3', 'UR.4', 'UR.5', 'UR.6', 'UR.8'], axis=1)
    money = state['Median Household Income 2018']
    incomes = []
    for income in money:
        incomes.append(float(income.split(',')[0][1:] + income.split(',')[1]))
    state['Median Household Income 2018'] = incomes
    return state
    
joinedWY2 = preprocess2(WY.copy(), df.copy())
joinedWA2 = preprocess2(WA.copy(), df.copy())
joinedVA2 = preprocess2(VA.copy(), df.copy())
joinedOH2 = preprocess2(OH.copy(), df.copy())

# fig, axs = plt.subplots(2,2, figsize=(10,10))
# axs[0,0].scatter(joinedOH2['Median Household Income 2018'][1:], joinedOH2['Mortality'][1:])
# axs[0,0].set_title("Ohio")
# axs[0,1].scatter(joinedVA2['Median Household Income 2018'][1:], joinedVA2['Mortality'][1:])
# axs[0,1].set_title("Virginia")
# axs[1,0].scatter(joinedWA2['Median Household Income 2018'][1:], joinedWA2['Mortality'][1:])
# axs[1,0].set_title("Washington")
# axs[1,1].scatter(joinedWY2['Median Household Income 2018'][1:], joinedWY2['Mortality'][1:])
# axs[1,1].set_title("Wyoming")

# for ax in axs.flat:
#     ax.set(xlabel="Median Household Income (2018)", ylabel="Mortality")

In [23]:
def trends(us, state):
    us = us[us['state'] == state]
    us = us.drop(['county', 'fips'], axis=1)
    dates = us['date'].unique()
    cases = []
    deaths = []
    for date in dates:
        temp = us[us['date'] == date]
        cases.append(sum(temp['cases']))
        deaths.append(sum(temp['deaths']))
    us = pd.DataFrame()
    base = datetime.datetime(2020, int(dates[0][-5:-3]), int(dates[0][-2:]))
    dates =  np.array([base + datetime.timedelta(hours=(24 * i)) for i in range(len(dates))])
    us['dates'] = dates
    us['cases'] = np.array(cases)
    us['deaths'] = np.array(deaths)
    return us
trendsWY = trends(df.copy(), 'Wyoming')
# fig, ax = plt.subplots(1, 2, constrained_layout=True, figsize=(12, 6))
# ax[0].plot(trendsWY['dates'], trendsWY['cases'])
# ax[1].plot(trendsWY['dates'], trendsWY['deaths'])
# ax[0].set_xlim((np.datetime64('2020-03-07'), np.datetime64('2020-11-26')))
# ax[1].set_xlim((np.datetime64('2020-03-07'), np.datetime64('2020-11-26')))
# for label in ax[0].get_xticklabels():
#         label.set_rotation(40)
#         label.set_horizontalalignment('right')
# for label in ax[1].get_xticklabels():
#         label.set_rotation(40)
#         label.set_horizontalalignment('right')
# ax[0].set(xlabel="Date", ylabel="Cases")
# ax[1].set(xlabel="Date", ylabel="Deaths")
# ax[0].set_title("Wyoming")
# ax[1].set_title("Wyoming")

In [24]:
def summary(us):
    states = us['state'].unique()
    points = []
    for state in states:
        trend = trends(us.copy(), state)
        rate = trend['cases'][len(trend)-1] - trend['cases'][len(trend)-5]
        points.append(np.sum(rate/5))
    return sorted(points)

data = summary(df.copy())
# fig, axs = plt.subplots(1, 2, figsize=(10, 6))
# axs[0].violinplot(data, showmedians=True, showmeans = True)
# axs[0].set(ylabel="Cases/Day")
# axs[0].set_title("Distribution of States by 5 day Case Rate ")
# axs[1].boxplot(data)
# axs[1].set(ylabel="Cases/Day")
# axs[1].set_title("Distribution of States by 5 day Case Rate ")
# sumdf = pd.DataFrame(data)
# sumdf.describe()

,0
count,55.000000
mean,2331.683636
std,2439.250186
min,0.000000
25%,746.100000
50%,1630.600000
75%,3101.600000
max,11497.000000


In [26]:
qq = []
res= []
i = 1
for point in data:
    qi = (i - 0.5)/55
    expo = np.log(1 - qi)/-0.000409 #Derived from Var(x) lambda = -0.000409
    qq.append(expo)
    i += 1
# plt.scatter(qq, data)
# plt.plot([0,12000],[0,12000])
# plt.xlabel("Cases/Day")
# plt.ylabel("Cases/Day")
# plt.title("QQ Plot for Cases/Day")

In [28]:
def ml_preprocess(dframe, us, code, state):
    name = []
    county = []
    for i in range(1555):
        county.append(dframe['County'][i].iloc[0][0:-11])
        county.append(dframe['County'][i].iloc[1][0:-11])
        name.append(dframe['County'][i].iloc[0][-2:])
        name.append(dframe['County'][i].iloc[1][-2:])
    dframe['code'] = np.array(name)
    dframe['County'] = np.array(county)
    us = us[us['state'] == state]
    dframe = dframe[dframe['code'] == code]
    cases=[]
    for county in range(len(dframe.index)):
        fips = dframe['County'].iloc[county]
        cases.append(sum(us[us['county'] == fips]['cases']))
    dframe['cases'] = np.array(cases)    
    return dframe
oh = ml_preprocess(df5.copy(), df.copy(), 'OH', 'Ohio')
va = ml_preprocess(df5.copy(), df.copy(), 'VA', 'Virginia')
wa = ml_preprocess(df5.copy(), df.copy(), 'WA', 'Washington')
wy = ml_preprocess(df5.copy(), df.copy(), 'WY', 'Wyoming')

# fig, axs = plt.subplots(2,2, figsize = (12,10))
# axs[0,0].scatter(wa['MigraRate'], wa['cases'])
# axs[0,1].scatter(wa['DeathRate'], wa['cases'])
# axs[1,0].scatter(wy['MigraRate'], wy['cases'])
# axs[1,1].scatter(wy['BirthRate'], wy['cases'])
# lst = ["MigraRate", "DeathRate", "MigraRate", "BirthRate"]
# lst2 = ["Washington Cases vs ", "Washington Cases vs ", "Wyoming Cases vs ", "Wyoming Cases vs "]
# for i, ax in enumerate(axs.flat):
#     ax.set(xlabel=lst[i], ylabel="Cases")
#     ax.set_title(lst2[i] + lst[i])
#wy.corr()